In [1]:
from huggingface_hub import login

login()

In [ ]:
model_path = "model/gemma-3-ib-it"

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

datasets= load_dataset("json", data_files = "datasets.json") 
datasets = datasets["train"].train_test_split(test_size = 0.25)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, attn_implementation='eager')
if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

model_name = model_path.split("/")[-1]

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    target_modules
     = ["q_proj", "v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

def preprocess_function(examples):
    texts = []
    for conv in examples["messages"]:
        text = "\n".join([msg["role"] + ": " + msg["content"] for msg in conv])
        texts.append(text)
    model_inputs = tokenizer(texts, max_length=512, truncation=True, padding = "max_length")
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)


training_args = TrainingArguments(
    "trained_model",
    eval_strategy = "epoch",
    learning_rate=2e-4,
    save_strategy="no",
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs=5,
    report_to="none",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1
)

Map:   0%|          | 0/12991 [00:00<?, ? examples/s]

Map:   0%|          | 0/4331 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
)

torch.cuda.empty_cache()
trainer.train()

# trainer.push_to_hub()
trainer.save_model("model/trained_model")
tokenizer.save_pretrained("model/trained_model")

NameError: name 'Trainer' is not defined

In [4]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

NameError: name 'trainer' is not defined

In [6]:
model_path = "model\\trained_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

text = input()

messages = [
    {"role" : "user", "content" : text},
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'model\trained_model'.